In [39]:
data = Import["DataScience_salaries_2023-2025.csv", "CSV"];

In [40]:
graph = Graph[UndirectedEdge @@@ data];

In [41]:
partition[graph_] := 
    Module[{list, listA, listB, subList, added},
        list = VertexList[graph];
        listA = {list[[1]]};
        listB = AdjacencyList[graph, list[[1]]];

        (* Itera sobre los vértices restantes del grafo *)
        For[i = 2, i <= Length[list], i++, 
            If[!MemberQ[Join[listA, listB], list[[i]]],
                subList = AdjacencyList[graph, list[[i]]];
                added = False;
                (* Itera sobre los vecinos del vértice actual *)
                For[j = 1, j <= Length[subList] && !added, j++,
                    (* Si uno de los vecinos está en la lista A,
                    añade el vértice a la lista B *)
                    If[MemberQ[listA, subList[[j]]], 
                        AppendTo[listB, list[[i]]]; 
                        added = True
                    ];
                    (* Si uno de los vecinos está en la lista B,
                    añade el vértice a la lista A *)
                    If[MemberQ[listB, subList[[j]]], 
                        AppendTo[listA, list[[i]]]; 
                        added = True
                    ]
                ];
                (* Si no se ha añadido el vértice a ninguna lista,
                lo añade a la lista A por defecto *)
                If[!added,
                    AppendTo[listA, list[[i]]]
                ]
            ]
        ];   
        {listA, listB}
    ]
(* Función que verifica si un grafo es bipartito y retorna la 
partición de conjuntos si es así *)
partitionedSets[graph_] :=
    Module[{partitions},
        If[BipartiteGraphQ[graph],
            partitions = partition[graph];
            Print["la longitud de los conjuntos particionados son:"];
            Print["Para U: ", Length[partitions[[1]]], " y Para V: ", Length[partitions[[2]]]];
            Print["Algunos datos son:"];
            Print["Para U:  ", RandomSample[partitions[[1]],5]];
            Print["Para V:  ", RandomSample[partitions[[2]],5]],
            "No es un grafo bipartito"
        ]
    ]
projection[graph_, setUorV_] :=
    Module[{projectedGraph, set, list1, list2},
        set = Switch[setUorV,
            U, partition[graph][[1]],
            V, partition[graph][[2]]
        ];
        projectedGraph = {};

        (* Itera sobre cada vértice en el conjunto seleccionado (set). *)
        For[i = 1, i <= Length[set], i++,
        list1 = AdjacencyList[graph, set[[i]]];
            (* Para cada vértice j en el conjunto set, verifica si existe una intersección 
            entre los vecinos de los vértices i y j. *)
            For[j = 1, j <= Length[set], j++,
                If[i != j,
                   list2 = AdjacencyList[graph, set[[j]]];
                   (* Si los conjuntos de adyacencia de i y j tienen elementos en común (hay una intersección) 
                   y aún no se ha agregado la conexión (arista) entre ellos, se añade al grafo proyectado. *)
                   If[Intersection[list1, list2] != {} && !EdgeQ[projectedGraph, set[[j]] <-> set[[i]]],
                       AppendTo[projectedGraph, set[[i]] <-> set[[j]]]
                   ]
                ]
            ]
        ];
        projectedGraph
    ]

projections[graph_, setUorV_] := 
    If[BipartiteGraphQ[graph],
        (* Según el valor de setUorV, calcula la proyección del conjunto U o V sobre el conjunto contrario. *)
        Switch[setUorV,
            U, Print["Un subgrafo de la proyeccion del conjunto U sobre V es: "]; 
            Graph[RandomSample[projection[graph,U], 100], GraphLayout -> "SpringEmbedding"],
            V, Print["Un subgrafo de la proyeccion del conjunto V sobre U es: "]; 
            Graph[RandomSample[projection[graph,V], 100], GraphLayout -> "SpringEmbedding"],
            _, "Conjunto Invalido"
        ],
        "No es un grafo bipartito"
    ]

In [46]:
partitionedSets[graph]

la longitud de los conjuntos particionados son:
Para U: 308 y Para V: 9246
Algunos datos son:
Para U:  {Machine Learning Developer, Data Strategist, 
 
>    Business Insights Manager, Autonomous Vehicle Technician, 
 
>    Applied Data Scientist}
Para V:  {241700, 172343, 44294, 125800, 165928}


In [47]:
graphU = projection[graph, U];

In [50]:
Length[VertexList[graphU]]

295